In [ ]:
# 递归定义法，f(n)=g(f(n-1))要包括base case和recursive case，更为简洁但运行效率比循环差。如果递归式为f(n)=g(f(n-1))则base case应定义n的最小值；如果递归式为f(n)=g(f(n+1))则base case应定义n的最大值。函数从n迭代到base case。
def factorial_recursion(n): #f(n)=n!
    # base case
    if n == 1:
        return 1
    # recursive case  f(n)=n∗f(n−1)
    else:
        return n * factorial_recursion(n-1)   #在定义过程中调用自身  
factorial_recursion(4)

#二叉树模型
from math import exp, sqrt
class CRR(object):
    """
    定义CRR类，实现美式期权定价
    param S：基础资产初始价格
    param X：期权的行权价格
    param r：无风险利率
    param q：基础资产分红率
    param T：期权到期时间，单位为年，每年按365天计算
    param sigma：基础资产波动率
    param N：整个时间段分为N步
    """
    def __init__(self, S, X, r, q, T, sigma, N, opt_type='c'):
        self.S = S
        self.X = X
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.N = N
        self.opt_type = opt_type.lower()[0]       # 期权类型，put/call

        self.dt = self.T / self.N                 # 步长
        self.u = exp(self.sigma * sqrt(self.dt))  # 上涨系数
        self.d = 1 / self.u                       # 下跌系数
        self.p = (exp((self.r - self.q) * self.dt) - self.d) / (self.u - self.d)   # 上涨概率
        self.opt_spot = dict()                    # 存储期权spot_price, 加快迭代速度

    # private method
    def __maturity_price(self, n, nu): #双下划线开头但没有下划线结尾用于定义私有变量或私有方法，只能在类的内部进行使用
        """
        计算到期时，n步中有nu步上涨的情况下，基础资产的价格
        :param nu: n步中上涨的步数
        :return: 返回到期时基础资产的价格
        """
        return self.S * (self.u ** (2 * nu - n)) #即Sunudn-nu=Su2*nu-n

    def __binomial_pricing(self, n, nu):
        """
        使用递归法计算二叉树节点上期权的价值
        """
        # 如果(n, nu)组合已经计算过，则直接从字典中提取计算结果，否则执行计算过程
        if (n, nu) in self.opt_spot:
            return self.opt_spot[(n, nu)]

        # Compute the exercise profit
        stockPrice = self.__maturity_price(n, nu) #计算标的价格
        if self.opt_type == 'c':  #计算期权的内在价值(行权价值)
            exerciseProfit = max(0, stockPrice - self.X)
        else:
            exerciseProfit = max(0, self.X - stockPrice)

        # Base case
        if n == self.N: #定义第N步的期权价值为行权价值(此时到期没有持有价值，只有行权价值)
            return exerciseProfit

        # Recursive case: compute the binomial value
        discount_factor = exp(-self.r * self.dt)
        V_exp = self.p * self.__binomial_pricing(n + 1, nu + 1) + (1 - self.p) * self.__binomial_pricing(n + 1, nu) #即f(n, nu)=p*f(n+1, nu+1)+(1-p)*f(n+1, nu)
        binomial = discount_factor * V_exp #将n+1时刻的期权价值的期望折现到n时刻得到n时刻的持有价值

        # 计算美式期权的价值
        opt_amr = max(binomial, exerciseProfit) #持有价值和行权价值取更大的作为期权价值
        self.opt_spot[(n, nu)] = opt_amr #将期权价值保存到opt_spot字典里
        return opt_amr

    def pricing(self):
        """使用CRR模型计算期权价格"""
        return self.__binomial_pricing(0,0)

#实际计算
T = 15 / 365  # time to maturity
N = 100  # number of steps
S = 2.4  # initial asset price
r = 0.0246  # risk free rate
q = 0.0  # market dividend rate
X = 2.45  # strike price
sigma = 0.27444
opt_type = 'p'

model = CRR(S, X, r, q, T, sigma, N, opt_type)
price = model.pricing()
print(price)

#计算基础资产波动率
from datetime import datetime
import pandas_datareader.data as web

start = datetime(2017, 1, 1)
# end = datetime(2017, 5, 24)
aapl = web.DataReader('AAPL', 'iex', start)    # 数据源：iex
aapl.tail()

aapl['rtn'] = aapl.close.pct_change()                           # 日收益率
aapl['volatility'] = aapl.rtn.rolling(100).std() * sqrt(252)     # 年化波动率

#定价
T = 34 / 365  # time to maturity
N = 100  # number of steps
S = 170.84  # initial asset price
r = 0.0236  # risk free rate
q = 0.0165  # market dividend rate
X = 170  # strike price
sigma = aapl.volatility[-1]
# sigma = 0.312
opt_type = 'p'

model = CRR(S, X, r, q, T, sigma, N, opt_type)
price = model.pricing()
print(price)